In [1]:
import pandas as pd
import statsmodels.api as sm
import statsmodels
import seaborn as sns
import numpy as np
project_path = "/mnt/vast/hpc/bvardarajan_lab/LPA_analysis"
version_name = "standardized_output"

pd.options.mode.chained_assignment = None 

Please note that all the table illustrations removed the index, the table should have subject index for aligning in actual running

# Encode the output from Coassin pipeline

In [2]:
from lpa_pipeline import encodings
print(encodings.__doc__)

The pipeline encodes outputs from coassin_pipeline

Encoding Rule:

* If raw/raw.txt file total coverage < ``raw_total_coverage_threshold``,
  encode that position "missing" for that person
* If total coverage >= ``raw_total_coverage_threshold``, in annotated file
    * If position is missing in annotated file, the variant is coded 0
    * If position is present in the annotated, if both:
        1. variant_level value > ``variant_level_threshold``
        2. the total reads supporting (variant_level*total_coverage)
           value >= ``read_supporting_threshold``,
      are met, the variant is coded 1, otherwise 0

Example:

    The class should be initiated as follows::

        eco = encodings.EncodingCoassinOutput(
            input_path = "/some/parent/path/of/bam/output" # or next line
            bam_list = "/paths/to/a/file/recording/bam/path/line/by/line.txt"
            output_path = "output_path"# required
            )

    The encoding process include an individual encodi

Using parent path:

In [3]:
eco = encodings.EncodingCoassinOutput(
    input_path=f"{project_path}/coassin_pipeline/pipeline_output",
    output_path=f"{project_path}/dataset/tidied_output",
    verbosity=0
    )

0it [00:00, ?it/s]

For txt file, the txt file should have each path in a row

In [ ]:
eco = encodings.EncodingCoassinOutput(
    bam_list = f"{project_path}/coassin_pipeline/data_inflow/bam_list_n=3.txt",
    output_path = f"{project_path}/dataset/tidied_output/illustration_run",
    verbosity = 1  # you can set to 0 to mute output
    )

In [ ]:
eco.encode_individual(saving_step = 1)

When encoding, the class will search all the output under output_path

In [ ]:
complete_coverage_total = eco.generate_coverage_total(save=True)
complete_encoded_result = eco.generate_encoded_results(save=True)

I suggest run this via SGE/SLURM rather than Jupyter, it took ~3-4 hrs at a sample size of ~4000

# Intersect with eigenstrat result 
Any other table with ethnicity information is fine

In [5]:
# the complete_encoded_result on the cell above
encoding_result = pd.read_csv(
    f"{project_path}/dataset/tidied_output/encoded_result_final.csv", 
    index_col = 0).T # One-hot-encoded table, Subject at index, snps on the header

# A ethnicity information table
# Any table provided a "ethnicity" column, this gives PCA result from eigenstrat as well
eigen_result = pd.read_csv(
    f"{project_path}/dataset/ethnicity_from_eigenstrat/eigenstrat_complete.csv", 
    index_col = 0) 
print(encoding_result.shape, eigen_result.shape)

(3915, 2130) (3819, 4)


The eigenstrat format is as follows, number of PCs doesn't matter:

In [6]:
# removed the index ID for illustration
eigen_result.head(3).reset_index(drop = True) 

,PC1,PC2,PC3,ethnicity
0,-0.0394,0.0118,-0.0001,EU
1,0.0280,0.0030,0.0480,EU
2,-0.0429,0.0254,-0.0240,EU


Align the encoding and ethnicity information

In [7]:
encoding_result_aligned, eigen_result_aligned = encoding_result.align(eigen_result, join = "inner", axis = 0)
print(encoding_result_aligned.shape, eigen_result_aligned.shape)

(3817, 2130) (3817, 4)


# Filter the encoded variants

In [8]:
from lpa_pipeline import snps_filter
filter_AB = snps_filter.SnpsFilter()
filtered_result, drop_mask, drop_report = filter_AB.filter_a_and_b(encoding_result_aligned)

In [9]:
drop_mask.head(3)

,filtered_A,filtered_B,filtered
snp_pos,,,
11-G/A,True,False,False
16-T/C,True,False,False
17-G/A,True,False,False


In [10]:
drop_report

,left,drop
filtered_A,1764,366
filtered_B,1533,597
filtered,1421,709


In [11]:
filtered_result.shape

(3817, 1421)

# Locus table illustration

In [12]:
from lpa_pipeline import locus_collector
print(locus_collector.__doc__)


Example::

    lc = locus_collector.LocusCollector(
        input_path = "/some/parent/path/of/bam/output" #choose this or next line
        bam_list = "/paths/to/a/file/recording/bam/path/line/by/line.txt")

    locus_table = lc.generate_locus_table()




In [13]:
lc = locus_collector.LocusCollector(
    input_path = f"{project_path}/coassin_pipeline/pipeline_output")

In [14]:
# this will take a while
locus_table = lc.generate_locus_table()

In [15]:
locus_table["mylocus"].value_counts()

large-I        1280
short-I         495
Exon421         165
Exon422         162
near_splice      91
3splice421        1
Name: mylocus, dtype: int64

In [16]:
locus_table_filtered = locus_table.loc[
    locus_table.index.isin(filtered_result.columns)]
locus_table_filtered.shape

(1421, 7)

# New Exon variant compared with Coassin table S9

This is just for novelty check, coassin_paper_exon is a pd.DataFrame with one column recording the previously discovered SNPs from Coassin et al.'s *A comprehensive map of single-base polymorphisms in the hypervariable LPA kringle IV type 2 copy number variation region* Supplemental table S9

In [17]:
coassin_paper_exon = pd.read_csv(
    f"{project_path}/dataset/Coassin_exon_from_excel_OCR_tidied.csv", 
    index_col = 0)

In [18]:
coassin_paper_exon.head(5)

,pos-ref/var
0,579-A/C
1,583-C/A
2,584-C/T
3,585-G/A
4,585-G/T


In [19]:
# find the Exons in our table
locus_exon = locus_table_filtered[locus_table_filtered["coding"] == "Exon"].index.drop_duplicates()
# find the variants in our exon but not in coassin's report
new = pd.DataFrame(locus_exon[~locus_exon.isin(coassin_paper_exon["pos-ref/var"])])
new.shape

(164, 1)

In [20]:
identified = pd.DataFrame(locus_exon[locus_exon.isin(coassin_paper_exon["pos-ref/var"])])
identified.shape

(103, 1)

In [21]:
locus_table_filtered["novel"] = ~locus_table_filtered.index.isin(coassin_paper_exon["pos-ref/var"])

# Freq and appearance

In [19]:
from lpa_pipeline import freq_table_generator
print(freq_table_generator.__doc__)

A generator computing relative frequency of SNP carrier by group

Common usage:

Given two pandas.DataFrame

* ``class_info_table`` has one have columns ``class_variable`` indicate the group
* ``one_hot_table`` has all one-hot-variables(SNPs encoding),

Compute the SNPs frequency in each class defined in <class_variable>:

    Initialize::

        ftg = freq_table_generator.FreqTableGenerator(
            threshold = 0.01
            encoding = {0: "Not Detected",
                        1: "Rare",
                        2: "Common"})

    Generate the table::

        freq_table = ftg.generate_freq_table(
            class_info_table = class_info_table,
            one_hot_table = one_hot_table,
            class_variable = "<class_variable>"
            class_variable_list = ["<class_name_1>","<class_name_2>",...]
            #if only need a part of <class_variable> column
            )

    If you need a rarity classification as columns as well::

        freq_table_with_rarity = 

In [20]:
#removed the index for illustration, the table should have index for aligning!
eigen_result_aligned.head(3).reset_index(drop = True) 

,PC1,PC2,PC3,ethnicity
0,-0.0113,0.0170,-0.0402,AF
1,-0.0414,0.0200,-0.0290,EU
2,0.0344,-0.0184,0.0465,AF


In [21]:
ftg = freq_table_generator.FreqTableGenerator()
freq_table_complete = ftg.generate_freq_table_with_rarity(
    one_hot_table = filtered_result,
    class_info_table = eigen_result_aligned,
    class_variable = "ethnicity"
    )

In [22]:
freq_table_complete.head(3)

,count_AF,total_AF_detected,total_AF_population,freq_AF,count_EU,total_EU_detected,total_EU_population,freq_EU,count_HISP,total_HISP_detected,total_HISP_population,freq_HISP,AF,EU,HISP
snp_pos,,,,,,,,,,,,,,,
21-G/A,346.0,988,1120,0.350202,388.0,815,886,0.476074,945.0,1769,1811,0.534200,Common,Common,Common
31-T/C,384.0,1009,1120,0.380575,403.0,824,886,0.489078,1015.0,1777,1811,0.571187,Common,Common,Common
33-T/C,1.0,1010,1120,0.000990,0.0,823,886,0.000000,0.0,1779,1811,0.000000,Rare,Not Detected,Not Detected


In [23]:
freq_table_complete["Novel"] = ~(freq_table_complete.index.isin(coassin_paper_exon["pos-ref/var"]))
freq_table_complete["Novel"].value_counts()

True     1318
False     103
Name: Novel, dtype: int64

For more details, check visualization_freqs.ipynb

# Data Loading for Associations and meta-analysis

In [22]:
from lpa_pipeline import association

In [23]:
personal_info = pd.read_csv(
    f"{project_path}/dataset/phenotypes/WHICAP_pheno_lpa_202306022.csv",
    index_col = 0)
personal_info.set_index("WES_ID", inplace= True)
personal_info = personal_info[
    (~personal_info["AGE"].isna()) & 
    (~personal_info["DEM03"].isna())]
# removed an outlier in Insulin (809 MIU/mL)
personal_info.loc["washei30170", "INSL02_2"] = pd.NA

## APOE4+

We defined the APOE4+ numerical value by the appearance of "4" in the genotype

In [24]:
personal_info["APOE4+"] = personal_info["APOE"].str.extract("(4+)").replace({"44": 2.0, "4": 1.0, pd.NA: 0.0})
personal_info.loc[personal_info["APOE"] == "NANA", "APOE4+"] = pd.NA

## Others

In [25]:
encoding_aligned, personal_info_aligned = filtered_result.align(personal_info, join = "inner", axis = 0)
eigen_aligned, personal_info_aligned = eigen_result_aligned.align(personal_info_aligned, join = "inner", axis = 0)
other_exogs = pd.concat(
    [personal_info_aligned[["DEM03", "AGE", "APOE4+"]], 
     eigen_aligned],
    axis = 1,
    join = "inner"
).rename(columns = {"DEM03": "GENDER"})
other_exogs["intercept"] = 1

Triglycerides, Insulin, and C-Peptide are natural logged for it's right-skewed distribution

In [26]:
phenotypes = personal_info_aligned[list(association.target_strategy().keys())]
phenotypes["INSL02_2"] = np.log(phenotypes["INSL02_2"].astype(pd.Float64Dtype()))
phenotypes["LIP03_B"] = np.log(phenotypes["LIP03_B"].astype(pd.Float64Dtype()))
phenotypes["INSL03_2"] = np.log(phenotypes["INSL03_2"].astype(pd.Float64Dtype()))

In [27]:
VNTR_prediction = pd.read_csv(
    f"{project_path}/VNTR_pipeline/analysis_results/estimate_KIV2_length_PSV/predicted_result_20240102_new_mask_simple.csv", 
    index_col = 0)

In [28]:
WHICAP_phenotype_legend = {
    "DIABETES": "History of Diabetes",
    "DEMENTIA": "Clinical Alzheimer's disease",
    "HEART": "History of Heart Disease",
    "HYPERTENSION": "Hypertension",
    "STROKE": "Stroke",
    "LIP01_B": "Cholesterol (mg/dL)",
    "LIP02_B": "HDL (mg/dL)",
    "LIP03_B": "Triglycerides (mg/dL), natural logged",
    "LIP04_2": "LDL (mg/dL)",
    "INSL01_2": "Glucose (mg/dL)",
    "INSL02_2": "Insulin (mIU/mL), natural logged",
    "INSL03_2": "C-Peptide (ng/mL), natural logged",
    "HBA1C_2": "Hemoglobin A1C (%)"
}

# Phenotypes vs SNP

## Traits ~ SNP (meta-analysis)

In [29]:
output_name = f"{version_name}_trait_vs_SNP"

In [30]:
other_exogs.head().reset_index(drop = True)

,GENDER,AGE,APOE4+,PC1,PC2,PC3,ethnicity,intercept
0,0,81.0,0.0,-0.0113,0.0170,-0.0402,AF,1
1,1,77.0,0.0,-0.0414,0.0200,-0.0290,EU,1
2,1,71.0,1.0,0.0344,-0.0184,0.0465,AF,1
3,1,74.0,1.0,0.0221,0.0004,-0.0222,EU,1
4,1,68.0,0.0,-0.0367,0.0121,0.0452,EU,1


In [47]:
# head has too many NAs, using tails
phenotypes.tail().reset_index(drop = True)

,STROKE,DEMENTIA,DIABETES,HEART,HYPERTENSION,LIP01_B,LIP02_B,LIP03_B,LIP04_2,INSL01_2,INSL02_2,INSL03_2,HBA1C_2
0,0.0,1.0,0,0,1,NaN,NaN,<NA>,95.0,83.0,2.332144,1.7613,6.37
1,0.0,2.0,0,1,1,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,NaN
2,0.0,2.0,0,0,1,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,NaN
3,0.0,2.0,0,0,1,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,NaN
4,0.0,1.0,0,1,1,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,NaN


In [20]:
snp_association = association.SNPAssociation()
snp_association.fit_transform(
    encoded_snp=encoding_aligned, 
    other_exogs=other_exogs,
    target_dataset=phenotypes,
    target_strategy=association.target_strategy(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    extra_iterate_on=["ethnicity"],
    snps_preprocessing_strategy=association.filter_c,
    snp_alias="variant",
    verbose=0
)

  0%|          | 0/13 [00:00<?, ?it/s]
                                                   
                                                    
                                                   
  8%|▊         | 1/13 [01:33<18:44, 93.75s/it][A
                                                   
                                                   
                                                   
 15%|█▌        | 2/13 [03:26<19:11, 104.67s/it]A
                                                   
                                                   
                                                   
 23%|██▎       | 3/13 [05:21<18:14, 109.47s/it]A
                                                   
                                                   
                                                   
 31%|███       | 4/13 [07:18<16:51, 112.38s/it]A
                                                   
                                                   
                                    

In [31]:
from lpa_pipeline import metal_toolkit
metal_pipeline_variant = metal_toolkit.METALToolkit(
    ethnicity=["EU", "AF", "HISP"],
    verbose=1,
    metal_path="/mnt/mfs/cluster/bin/METAL/metal",
    snp_alias="variant")

In [32]:
aggregate_results = metal_pipeline_variant.run_metal(
    path_association=f"{project_path}/data_analysis_result/association/{output_name}",
    path_meta=f"{project_path}/data_analysis_result/meta_analysis/{output_name}")

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

METAL scripts are saved to /mnt/vast/hpc/bvardarajan_lab/LPA_analysis/data_analysis_result/meta_analysis/standardized_output_trait_vs_SNP


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [33]:
from lpa_pipeline import post_processing
post_processor_variant = post_processing.PostProcessor(
    locus_table=locus_table_filtered,
    snp_alias="variant")

In [34]:
complete_output = post_processor_variant.post_process_meta_analysis(aggregate_results)
complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output.xlsx")

In [36]:
tidied_complete_output = post_processor_variant.clean_output(complete_output)
tidied_complete_output["trait"] = tidied_complete_output["trait"].apply(
    lambda x: WHICAP_phenotype_legend[x])
tidied_complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output_tidied.xlsx")

## Phenotypes ~ SNP + VNTR (meta-analysis)

In [37]:
output_name = f"{version_name}_trait_vs_SNP_adjusted_by_VNTR"

In [63]:
other_exogs_with_VNTR_estimate = pd.concat(
    [VNTR_prediction[["estimate"]], other_exogs],
    join = "inner",
    axis = 1
).rename(
    columns = {"estimate": "VNTR_estimate"}
)
other_exogs_with_VNTR_estimate.head().reset_index(drop = True)

,VNTR_estimate,GENDER,AGE,APOE4+,PC1,PC2,PC3,ethnicity,intercept
0,14.386291,1,75.0,0.0,0.0056,0.0716,-0.0368,AF,1
1,19.352792,1,66.0,0.0,-0.0269,-0.0093,0.0084,AF,1
2,18.638641,1,78.0,1.0,0.0156,-0.0250,0.0519,AF,1
3,18.183185,1,72.0,1.0,-0.0235,-0.0488,-0.0111,AF,1
4,21.130558,1,78.0,0.0,-0.0208,0.0224,0.0116,AF,1


In [32]:
snp_association = association.SNPAssociation()
snp_association.fit_transform(
    encoded_snp=encoding_aligned,
    other_exogs=other_exogs_with_VNTR_estimate,
    target_dataset=phenotypes,
    target_strategy=association.target_strategy(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    extra_iterate_on=["ethnicity"],
    snps_preprocessing_strategy=association.filter_c,
    snp_alias="variant",
    verbose=0
)

  0%|          | 0/13 [00:00<?, ?it/s]
                                                   
                                                   
                                                   
  8%|▊         | 1/13 [01:48<21:43, 108.65s/it]A
                                                   
                                                   
                                                   
 15%|█▌        | 2/13 [03:53<21:42, 118.45s/it]A
                                                   
                                                   
                                                   
 23%|██▎       | 3/13 [05:59<20:14, 121.48s/it]A
                                                   
                                                   
                                                   
 31%|███       | 4/13 [08:03<18:25, 122.78s/it]A
                                                   
                                                   
                                     

In [38]:
aggregate_results = metal_pipeline_variant.run_metal(
    path_association=f"{project_path}/data_analysis_result/association/{output_name}",
    path_meta=f"{project_path}/data_analysis_result/meta_analysis/{output_name}")
complete_output = post_processor_variant.post_process_meta_analysis(aggregate_results)
complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output.xlsx")
tidied_complete_output = post_processor_variant.clean_output(complete_output)
tidied_complete_output["trait"] = tidied_complete_output["trait"].apply(
    lambda x: WHICAP_phenotype_legend[x])
tidied_complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output_tidied.xlsx")

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

METAL scripts are saved to /mnt/vast/hpc/bvardarajan_lab/LPA_analysis/data_analysis_result/meta_analysis/standardized_output_trait_vs_SNP_adjusted_by_VNTR


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

# About APOE

## APOE ~ SNP (meta-analysis)

In [39]:
output_name = f"{version_name}_APOE_vs_SNP"

In [35]:
APOE_value = other_exogs[["APOE4+"]].rename(columns = {"APOE4+":"APOE4"})
other_exogs_no_APOE = other_exogs.drop(columns = "APOE4+")
other_exogs_no_APOE.head().reset_index(drop = True)

,GENDER,AGE,PC1,PC2,PC3,ethnicity,intercept
0,0,81.0,-0.0113,0.0170,-0.0402,AF,1
1,1,77.0,-0.0414,0.0200,-0.0290,EU,1
2,1,71.0,0.0344,-0.0184,0.0465,AF,1
3,1,74.0,0.0221,0.0004,-0.0222,EU,1
4,1,68.0,-0.0367,0.0121,0.0452,EU,1


In [36]:
APOE_strategy = {'APOE4': {'engine': statsmodels.regression.linear_model.OLS,
                           'preprocessing': association.dropna}}

In [37]:
snp_association = association.SNPAssociation()
snp_association.fit_transform(
    encoded_snp=encoding_aligned, 
    other_exogs=other_exogs_no_APOE,
    target_dataset=APOE_value,
    target_strategy=APOE_strategy,
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    extra_iterate_on=["ethnicity"],
    snps_preprocessing_strategy=association.filter_c,
    snp_alias="variant",
    verbose=0
)

  0%|          | 0/1 [00:00<?, ?it/s]
                                                   
                                                   
                                                   
100%|██████████| 1/1 [01:54<00:00, 114.43s/it][A


In [40]:
aggregate_results = metal_pipeline_variant.run_metal(
    path_association=f"{project_path}/data_analysis_result/association/{output_name}",
    path_meta=f"{project_path}/data_analysis_result/meta_analysis/{output_name}")
complete_output = post_processor_variant.post_process_meta_analysis(aggregate_results)
complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output.xlsx")
tidied_complete_output = post_processor_variant.clean_output(complete_output)
tidied_complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output_tidied.xlsx")

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

METAL scripts are saved to /mnt/vast/hpc/bvardarajan_lab/LPA_analysis/data_analysis_result/meta_analysis/standardized_output_APOE_vs_SNP


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Phenotypes ~ APOE (meta-analysis)

We didn't restrict the value of encoded to be binary throughout the pipeline, except the frequency computation. But we don't care freqeuncy for those numerical results. Thus, just give the APOE a placeholder value and we can re-use the same pipeline

In [39]:
output_name = f"{version_name}_trait_vs_APOE"

In [40]:
APOE_value = other_exogs[["APOE4+"]].rename(columns = {"APOE4+":"1-A/T"})
APOE_value.head(5)

,1-A/T
washei70623,0.0
washei70345,0.0
washei70493,1.0
washei71229,1.0
washei70625,0.0


In [41]:
other_exogs_no_APOE = other_exogs.drop(columns = "APOE4+")
other_exogs_no_APOE.head(5)

,GENDER,AGE,PC1,PC2,PC3,ethnicity,intercept
washei70623,0,81.0,-0.0113,0.0170,-0.0402,AF,1
washei70345,1,77.0,-0.0414,0.0200,-0.0290,EU,1
washei70493,1,71.0,0.0344,-0.0184,0.0465,AF,1
washei71229,1,74.0,0.0221,0.0004,-0.0222,EU,1
washei70625,1,68.0,-0.0367,0.0121,0.0452,EU,1


In [42]:
snp_association = association.SNPAssociation()
snp_association.fit_transform(
    encoded_snp=APOE_value, 
    other_exogs=other_exogs_no_APOE ,
    target_dataset=phenotypes,
    target_strategy=association.target_strategy(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    extra_iterate_on=["ethnicity"],
    snps_preprocessing_strategy=association.filter_c,
    snp_alias="APOE4",
    verbose=0
)

  0%|          | 0/13 [00:00<?, ?it/s]
                                     
                                     .65it/s]
  8%|▊         | 1/13 [00:00<00:03,  3.07it/s][A
                                     
                                     .74it/s]
 15%|█▌        | 2/13 [00:00<00:03,  3.06it/s][A
                                     
                                     .57it/s]
 23%|██▎       | 3/13 [00:00<00:03,  3.06it/s][A
                                     
                                     .52it/s]
 31%|███       | 4/13 [00:01<00:02,  3.04it/s][A
                                     
                                     .65it/s]
 38%|███▊      | 5/13 [00:01<00:02,  3.04it/s][A
                                     
                                     .79it/s]
 46%|████▌     | 6/13 [00:01<00:02,  3.11it/s][A
                                     
                                     .85it/s]
 54%|█████▍    | 7/13 [00:02<00:01,  3.15it/s][A
                              

In [43]:
metal_pipeline_APOE = metal_toolkit.METALToolkit(
    ethnicity = ["EU", "AF", "HISP"], 
    verbose = 1, 
    metal_path = "/mnt/mfs/cluster/bin/METAL/metal",
    snp_alias = "APOE4")
aggregate_results = metal_pipeline_APOE.run_metal(
    path_association=f"{project_path}/data_analysis_result/association/{output_name}",
    path_meta=f"{project_path}/data_analysis_result/meta_analysis/{output_name}")

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

METAL scripts are saved to /mnt/vast/hpc/bvardarajan_lab/LPA_analysis/data_analysis_result/meta_analysis/standardized_output_trait_vs_APOE


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [44]:
post_processor_APOE = post_processing.PostProcessor(
    locus_table = locus_table,
    snp_alias = "APOE4"
)
complete_output = post_processor_APOE.post_process_meta_analysis(aggregate_results, mode = "APOE4")
complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output.xlsx")
tidied_complete_output = post_processor_APOE.clean_output(complete_output)
tidied_complete_output["trait"] = tidied_complete_output["trait"].apply(
    lambda x: WHICAP_phenotype_legend[x])
tidied_complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output_tidied.xlsx")

# About existing serum level

## lpa/wIS ~ SNP + adjustments (association, N = 27)

In [41]:
output_name = f"{version_name}_serum_vs_SNP"

In [65]:
personal_info_serum_old = pd.read_csv(
    f"{project_path}/dataset/serum_data/serum_data_personal_info.csv")
personal_info_serum = pd.merge(
    left=personal_info.reset_index(),
    right=personal_info_serum_old[["WHICAP_ID"]], 
    left_on = "WHICAP_ID", 
    right_on = "WHICAP_ID"
).set_index("WES_ID")

In [66]:
other_exogs_serum = pd.concat(
    [personal_info_serum[["DEM03", "AGE", "APOE4+"]], 
     eigen_aligned], 
    axis = 1,
    join = "inner"
   ).rename(columns = {"DEM03": "GENDER"})
other_exogs_serum["intercept"] = 1

In [67]:
other_exogs_serum = other_exogs_serum.drop(columns = "ethnicity")

In [68]:
serum_result = pd.read_csv(
    f"{project_path}/dataset/serum_data/serum_data_serum.csv",
    index_col = 0
).rename(
    columns = {"LP(a) (nmol/L)": "lpa",
               "Isoform 1          (expression)": "isoform",
               "wAS": "wIS"})
serum_result = pd.merge(
    left = personal_info_serum.reset_index()[["WES_ID", "WHICAP_ID"]],
    right = serum_result,
    left_on = "WHICAP_ID",
    right_index = True
).set_index("WES_ID")
encoding_serum = encoding_aligned.loc[other_exogs_serum.index]

Lpa is a right-skewed data from it's nature, natural log it

In [69]:
serum_result["lpa"] = np.log(serum_result["lpa"])

In [70]:
other_exogs_serum.head().reset_index(drop = True)

,GENDER,AGE,APOE4+,PC1,PC2,PC3,intercept
0,1,74.00,0.0,-0.0435,0.0076,0.0145,1
1,1,93.00,0.0,0.0508,-0.0558,-0.0104,1
2,1,90.00,1.0,0.0390,0.0314,-0.0165,1
3,1,82.93,0.0,0.0043,0.0004,0.0135,1
4,0,93.72,0.0,0.0310,-0.0025,0.0136,1


In [71]:
serum_result.head().reset_index(drop = True)

,WHICAP_ID,lpa,Isoform 1 (KIV Motifs),Isoform 2 (KIV Motifs),Isoform 1 (% expression),Isoform 2 (% expression),isoform,Isoform 2 (expression),wIS
0,W05307,5.410663,17,27,0.86,0.14,0.86,0.14,18.40
1,W06744,4.202002,28,33,0.93,0.07,0.93,0.07,28.35
2,W06939,4.636184,18,-,1.00,-,1.00,-,18.00
3,W06984,2.510412,23,19,0.82,0.18,0.82,0.18,22.28
4,W07059,2.384165,17,27,0.72,0.28,0.72,0.28,19.80


In [72]:
serum_association = association.SNPAssociation()
serum_association.fit_transform(
    encoded_snp=encoding_serum, 
    other_exogs=other_exogs_serum,
    target_dataset=serum_result,
    target_strategy=association.target_strategy_serum(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    #extra_iterate_on = ["ethnicity"], we dont have enough sample to support regression by race
    snps_preprocessing_strategy=association.filter_c,
    snp_alias="variant",
    verbose=0
)

100%|██████████| 2/2 [00:21<00:00, 10.66s/it]       


In [42]:
wIS = pd.read_csv(
    f"{project_path}/data_analysis_result/association/{output_name}/wIS_OLS_N_snp=107.csv",
    index_col = 0)
complete_output = post_processor_variant.post_process_association(wIS)
complete_output.to_excel(f"{project_path}/data_analysis_result/association/{output_name}/wIS_complete_output.xlsx")
complete_output[[
    'index', 'variant Beta','FDR_adjusted_p-value', 'significant_FDR','r_squared',
    'mylocus', 'wt','mut', 'coding', 'novel']
].to_excel(f"{project_path}/data_analysis_result/association/{output_name}/wIS_complete_tidied.xlsx")

In [43]:
lpa = pd.read_csv(
    f"{project_path}/data_analysis_result/association/{output_name}/lpa_OLS_N_snp=107.csv", 
    index_col = 0)
complete_output = post_processor_variant.post_process_association(lpa)
complete_output.to_excel(f"{project_path}/data_analysis_result/association/{output_name}/lpa_complete_output.xlsx")
complete_output[[
    'index', 'variant Beta','FDR_adjusted_p-value', 'significant_FDR','r_squared',
    'mylocus', 'wt','mut', 'coding', 'novel']
].to_excel(f"{project_path}/data_analysis_result/association/{output_name}/lpa_complete_tidied.xlsx")

## lpa/wIS ~ SNPS + VNTR (association, N = 27)

In [44]:
output_name = f"{version_name}_serum_vs_SNP_adjusted_by_VNTR"

In [77]:
# same to the one above, you can skip this
personal_info_serum_old = pd.read_csv(
    f"{project_path}/dataset/serum_data/serum_data_personal_info.csv")
personal_info_serum = pd.merge(
    personal_info.reset_index(),
    personal_info_serum_old[["WHICAP_ID"]], 
    left_on = "WHICAP_ID", right_on = "WHICAP_ID").set_index("WES_ID")
other_exogs_serum = pd.concat(
    [personal_info_serum[["DEM03", "AGE", "APOE4+"]], 
     eigen_aligned], 
    axis = 1,
    join = "inner"
   ).rename(columns = {"DEM03": "GENDER"})
serum_result = pd.read_csv(
    f"{project_path}/dataset/serum_data/serum_data_serum.csv",
    index_col = 0).rename(
    columns = {"LP(a) (nmol/L)": "lpa",
               "Isoform 1          (expression)": "isoform",
               "wAS": "wIS"})
serum_result = pd.merge(
    left = personal_info_serum.reset_index()[["WES_ID","WHICAP_ID"]],
    right = serum_result,
    left_on = "WHICAP_ID",
    right_index = True
    ).set_index("WES_ID")
serum_result["lpa"] = np.log(serum_result["lpa"])

In [78]:
other_exogs_serum = pd.concat(
    [VNTR_prediction[["estimate"]], 
     other_exogs_serum], 
    axis = 1, join = "inner")
other_exogs_serum["intercept"] = 1
other_exogs_serum = other_exogs_serum.drop(columns = "ethnicity")

In [79]:
encoding_serum = encoding_aligned.loc[other_exogs_serum.index]

In [80]:
other_exogs_serum.head(5)

,estimate,GENDER,AGE,APOE4+,PC1,PC2,PC3,intercept
washei64289,20.135733,1,90.00,1.0,0.0390,0.0314,-0.0165,1
washei41440,18.623412,1,74.00,0.0,-0.0435,0.0076,0.0145,1
washei40407,22.622735,1,85.89,1.0,-0.0153,-0.0746,-0.0053,1
washei39055,16.639620,1,86.00,1.0,0.0186,-0.0413,0.0168,1
washei37339,17.739429,1,78.00,1.0,-0.0435,0.0052,-0.0327,1


In [81]:
serum_result["lpa"] = np.log(serum_result["lpa"])

In [82]:
asso = association.SNPAssociation()
asso.fit_transform(
    encoded_snp = encoding_serum, 
    other_exogs = other_exogs_serum,
    target_dataset = serum_result,
    target_strategy = association.target_strategy_serum(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    #extra_iterate_on = ["ethnicity"],
    snps_preprocessing_strategy = association.filter_c,
    snp_alias="variant",
    verbose = 0
)

100%|██████████| 2/2 [00:24<00:00, 12.09s/it]       


In [45]:
wIS = pd.read_csv(
    f"{project_path}/data_analysis_result/association/{output_name}/wIS_OLS_N_snp=107.csv",
    index_col = 0)
complete_output = post_processor_variant.post_process_association(wIS)
complete_output.to_excel(f"{project_path}/data_analysis_result/association/{output_name}/wIS_complete_output.xlsx")
complete_output[[
    'index', 'variant Beta', 'FDR_adjusted_p-value',
    'significant_FDR', 'r_squared',
    'mylocus', 'wt', 'mut', 'coding', 'novel']
].to_excel(f"{project_path}/data_analysis_result/association/{output_name}/wIS_complete_tidied.xlsx")

In [46]:
lpa = pd.read_csv(
    f"{project_path}/data_analysis_result/association/{output_name}/lpa_OLS_N_snp=107.csv", 
    index_col = 0)
complete_output = post_processor_variant.post_process_association(lpa)
complete_output.to_excel(f"{project_path}/data_analysis_result/association/{output_name}/lpa_complete_output.xlsx")
complete_output[[
    'index', 'variant Beta', 'FDR_adjusted_p-value',
    'significant_FDR', 'r_squared',
    'mylocus', 'wt', 'mut', 'coding', 'novel']
].to_excel(f"{project_path}/data_analysis_result/association/{output_name}/lpa_complete_tidied.xlsx")

# About VNTR

## VNTR ~ SNP + Adjustment (meta-analysis, N = 3774)

In [47]:
output_name = f"{version_name}_VNTR_vs_SNP"

In [86]:
VNTR_prediction = pd.read_csv(
    f"{project_path}/VNTR_pipeline/analysis_results/estimate_KIV2_length_PSV/predicted_result_20240102_new_mask_simple.csv",
    index_col = 0
)
VNTR_prediction = VNTR_prediction.loc[
    phenotypes.index,["estimate"]
].rename(
    columns = {"estimate": "VNTR_estimate"}
)

In [87]:
VNTR_prediction.head()

,VNTR_estimate
washei70623,17.322922
washei70345,18.421054
washei70493,19.154933
washei71229,14.107403
washei70625,22.170373


In [88]:
other_exogs.head()

,GENDER,AGE,APOE4+,PC1,PC2,PC3,ethnicity,intercept
washei70623,0,81.0,0.0,-0.0113,0.0170,-0.0402,AF,1
washei70345,1,77.0,0.0,-0.0414,0.0200,-0.0290,EU,1
washei70493,1,71.0,1.0,0.0344,-0.0184,0.0465,AF,1
washei71229,1,74.0,1.0,0.0221,0.0004,-0.0222,EU,1
washei70625,1,68.0,0.0,-0.0367,0.0121,0.0452,EU,1


In [89]:
VNTR_strategy = {"VNTR_estimate": {"engine": sm.OLS,
                                   "preprocessing": association.dropna}}

In [90]:
asso = association.SNPAssociation()
asso.fit_transform(
    encoded_snp=encoding_aligned, 
    other_exogs=other_exogs,
    target_dataset=VNTR_prediction,
    target_strategy=VNTR_strategy,
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    extra_iterate_on=["ethnicity"],
    snps_preprocessing_strategy=association.filter_c,
    snp_alias="variant",
    verbose=0
)

  0%|          | 0/1 [00:00<?, ?it/s]
                                                   
                                                   
                                                   
100%|██████████| 1/1 [02:15<00:00, 135.87s/it][A


In [48]:
aggregate_results = metal_pipeline_variant.run_metal(
    path_association=f"{project_path}/data_analysis_result/association/{output_name}",
    path_meta=f"{project_path}/data_analysis_result/meta_analysis/{output_name}")
complete_output = post_processor_variant.post_process_meta_analysis(aggregate_results)
complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output.xlsx")
tidied_complete_output = post_processor_variant.clean_output(complete_output)
tidied_complete_output.to_excel(
    f"{project_path}/data_analysis_result/meta_analysis/{output_name}/complete_output_tidied.xlsx")

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

METAL scripts are saved to /mnt/vast/hpc/bvardarajan_lab/LPA_analysis/data_analysis_result/meta_analysis/standardized_output_VNTR_vs_SNP


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

## Phenotypes ~ VNTR + Adjustment (meta-analysis N = 3774)

In [92]:
output_name = f"{version_name}_trait_vs_VNTR"

In [93]:
vntr_as_exog = VNTR_prediction.rename(columns = {"VNTR_estimate": "1-A/T"})

In [94]:
vntr_as_exog.head()

,1-A/T
washei70623,17.322922
washei70345,18.421054
washei70493,19.154933
washei71229,14.107403
washei70625,22.170373


In [95]:
other_exogs.head()

,GENDER,AGE,APOE4+,PC1,PC2,PC3,ethnicity,intercept
washei70623,0,81.0,0.0,-0.0113,0.0170,-0.0402,AF,1
washei70345,1,77.0,0.0,-0.0414,0.0200,-0.0290,EU,1
washei70493,1,71.0,1.0,0.0344,-0.0184,0.0465,AF,1
washei71229,1,74.0,1.0,0.0221,0.0004,-0.0222,EU,1
washei70625,1,68.0,0.0,-0.0367,0.0121,0.0452,EU,1


In [97]:
asso = association.SNPAssociation()
asso.fit_transform(
    encoded_snp=vntr_as_exog, 
    other_exogs=other_exogs,
    target_dataset=phenotypes,
    target_strategy=association.target_strategy(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    extra_iterate_on=["ethnicity"],
    snp_alias="VNTR_estimate",
    verbose=0
)

  0%|          | 0/13 [00:00<?, ?it/s]
                                     
                                     .80it/s]
                                     .71it/s]
  8%|▊         | 1/13 [00:00<00:04,  2.57it/s][A
                                     
                                     .85it/s]
                                     .76it/s]
 15%|█▌        | 2/13 [00:00<00:04,  2.55it/s][A
                                     
                                     .00it/s]
 23%|██▎       | 3/13 [00:01<00:03,  2.57it/s][A
                                     
                                     .78it/s]
 31%|███       | 4/13 [00:01<00:03,  2.55it/s][A
                                     
                                     .29it/s]
                                     .19it/s]
 38%|███▊      | 5/13 [00:01<00:03,  2.50it/s][A
                                     
                                     .98it/s]
 46%|████▌     | 6/13 [00:02<00:02,  2.59it/s][A
                         

In [98]:
metal_pipeline_vntr = metal_toolkit.METALToolkit(
    ethnicity = ["EU", "AF", "HISP"], 
    verbose = 1, 
    metal_path = "/mnt/mfs/cluster/bin/METAL/metal",
    snp_alias = "VNTR_estimate")

In [99]:
aggregate_results = metal_pipeline_vntr.run_metal(
    path_association=f"{project_path}/data_analysis_result/association/{output_name}",
    path_meta=f"{project_path}/data_analysis_result/meta_analysis/{output_name}")

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

METAL scripts are saved to /mnt/vast/hpc/bvardarajan_lab/LPA_analysis/data_analysis_result/meta_analysis/standardized_output_trait_vs_VNTR


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

## LPA/WIS ~ VNTR + adjustment (Association N = 27)

In [102]:
output_name = f"{version_name}_serum_vs_VNTR"

In [103]:
vntr_as_exog_serum = vntr_as_exog.loc[encoding_serum.index]

In [104]:
vntr_as_exog_serum.head()

,1-A/T
washei64289,20.135733
washei41440,18.623412
washei40407,22.622735
washei39055,16.639620
washei37339,17.739429


In [105]:
other_exogs_serum = other_exogs_serum.drop(columns = "estimate")
other_exogs_serum.head()

,GENDER,AGE,APOE4+,PC1,PC2,PC3,intercept
washei64289,1,90.00,1.0,0.0390,0.0314,-0.0165,1
washei41440,1,74.00,0.0,-0.0435,0.0076,0.0145,1
washei40407,1,85.89,1.0,-0.0153,-0.0746,-0.0053,1
washei39055,1,86.00,1.0,0.0186,-0.0413,0.0168,1
washei37339,1,78.00,1.0,-0.0435,0.0052,-0.0327,1


In [106]:
asso.fit_transform(
    encoded_snp=vntr_as_exog_serum, 
    other_exogs=other_exogs_serum,
    target_dataset=serum_result,
    target_strategy=association.target_strategy_serum(),
    output_path=f"{project_path}/data_analysis_result/association/{output_name}",
    snp_alias="VNTR_estimate",
    verbose=0
)

100%|██████████| 2/2 [00:00<00:00,  5.71it/s]
